In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
import time
from datetime import timedelta
from datetime import datetime
from torchsummary import summary

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels, averageGradients
from torch.utils.tensorboard import SummaryWriter
import math

In [2]:
class Arguments():
    def __init__(self):
        self.images = 3012
        self.clients = 3
        self.rounds = 1000
        self.epochs = 1
        self.local_batches = 20
        self.lr = 0.02
        self.dropout1 = 0.25
        self.dropout2 = 0.5
        self.C = 0.66
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = True
        self.save_model = True
        self.save_model_interval = 200

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


cuda


In [3]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

print(clients)
print("number of clients : ", len(clients))

[{'hook': <VirtualWorker id:client1 #objects:0>}, {'hook': <VirtualWorker id:client2 #objects:0>}, {'hook': <VirtualWorker id:client3 #objects:0>}]
number of clients :  3


In [4]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [5]:
print(len(global_train))
print(type(global_train))
print(len(global_test))
print(type(global_test))
print(len(train_group))
print(type(train_group))
print(len(test_group))
print(type(test_group))

3012
<class 'FLDataset.CovidDataset'>
753
<class 'FLDataset.CovidDataset'>
3
<class 'dict'>
3
<class 'dict'>


In [6]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    print("len(client", str(inx), "train set) = ", len(trainset_ind_list))
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
    client['samples'] = len(trainset_ind_list) / args.images

len(client 0 train set) =  1004
len(client 1 train set) =  1004
len(client 2 train set) =  1004


In [7]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = CovidDataset('./test.csv', transform=transforms.Compose([Rescale(32), ToTensor()]))
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                               out_channels = 32,
                               kernel_size = 3,
                               stride = 1)
        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1)
        self.fc1 = nn.Linear(14*14*64, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, p=args.dropout1)
        x = x.view(-1, 14*14*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=args.dropout2)
        x = self.fc2(x)
        return F.softmax(x)

In [9]:
def ClientUpdate(args, device, client):
#     print("client: ", client)
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
#             output = client['model'](data.float())
#             loss = F.nll_loss(output, target.squeeze(1))
            output = client['model'](data)
            loss = client['criterion'](output, target.squeeze(1))
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
#             print("output: ", output)
#             print("target squeeze: ", target.squeeze(1))
#             print("loss: ", loss)
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            loss.backward()
            client['optim'].step()
            
            
            if batch_idx % args.log_interval == 0 or batch_idx==len(client['trainset'])-1:
#                 loss = loss.get() 
                print('Model [{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, (batch_idx+1) * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * (batch_idx+1) / len(client['trainset']), loss.item()/args.log_interval))
                
#     client['model'].get() 

In [10]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for d in test_loader:
            data = d['image']
            target = d['label']
            data, target = data.to(device), target.to(device)
            if(str(device)=='cuda'):
                model.cuda()
            output = model(data.float())
#             test_loss += F.nll_loss(output, target.squeeze(1), reduction='sum').item() # sum up batch loss
            loss_fn = nn.CrossEntropyLoss(reduction='sum')
            test_loss += loss_fn(output, target.squeeze(1)).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [11]:
writer = SummaryWriter()

In [12]:
torch.manual_seed(args.torch_seed)
global_model = Net().to(device)
summary(global_model, (1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Linear-3                  [-1, 128]       1,605,760
            Linear-4                    [-1, 3]             387
Total params: 1,624,963
Trainable params: 1,624,963
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.60
Params size (MB): 6.20
Estimated Total Size (MB): 6.81
----------------------------------------------------------------


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:414: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  response = command_method(*args_, **kwargs_)


In [13]:
# training
for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)
    client['criterion'] = nn.CrossEntropyLoss(reduction='mean')
#     client['pengine'] = PrivacyEngine(
#                                        client['model'],
#                                        batch_size=args.local_batches,
#                                        sample_size=len(client['trainset']),
#                                        alphas=range(2,32),
#                                        noise_multiplier=0,
#                                        max_grad_norm=1000
#                                     )
#     client['pengine'].attach(client['optim']) 
    
# start training model
training_start_time = time.time()
for fed_round in range(args.rounds):
    print("")
    print("===================================================================")
    print("[round] = ", fed_round+1, "/", args.rounds)
    print("===================================================================")
    
    round_train_start_time = time.time()
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))    
    
    # number of selected clients
    m = int(max(math.ceil(args.C * args.clients), 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    active_clients = selected_clients
    #np.random.seed(fed_round)
    #active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    #active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    client_cnt = 0
    for client in active_clients:
        print("* [client count] = ", client_cnt+1 , "/", len(active_clients))
        client_train_start_time = time.time()
        ClientUpdate(args, device, client)
        client_cnt += 1
        client_train_time = round(time.time()-client_train_start_time)
        print("* [client_train_time] = ", str(timedelta(seconds=(client_train_time))))
        print("---------------------------------------------------------------")
    
#         # Testing 
#         for client in active_clients:
#             test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    acc = test(args, global_model, device, global_test_loader, 'Global')
    writer.add_scalar("Accuracy/train", acc, fed_round)
    writer.flush()
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
    # training time per round
    total_train_time = round(time.time()-training_start_time)
    round_train_time = round(time.time()-round_train_start_time)
    print("** [total train time]: ", str(timedelta(seconds=total_train_time)))
    print("** [round train time]: ", str(timedelta(seconds=round_train_time)))
    
    if (args.save_model and fed_round%args.save_model_interval==0 and fed_round!=0):
        now = datetime.now() 
        date = now.strftime("%Y_%m_%d_%H%M")
        torch.save(global_model.state_dict(), date + "_FedAvg_with_DP_round_" + str(fed_round) + ".pth")
        print("model saved : "+ date +"_FedAvg_with_DP_round_" + str(fed_round) + ".pth")


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)



[round] =  1 / 1000
* [client count] =  1 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.134135
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.120144
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.100142
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.110144
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.130144
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.130144
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.118786
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.100144
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.090144
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.115144
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.135144
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.105144
* [client_train_time] =  0:00:12
-----------------------------------------

Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.071598
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.092257
* [client_train_time] =  0:00:13
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.100773
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.102892
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.092978
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.093613
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.097594
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.104354
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9590, Accuracy: 610/753 (81%)

** [total train time]:  0:03:26
** [round train time]:  0:00:29

[round] =  8 / 1000
* [client count] =  1 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.093223
Model [client3] Train Epoch

Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.086422
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.074793
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.105050
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.8982, Accuracy: 612/753 (81%)

** [total train time]:  0:06:23
** [round train time]:  0:00:30

[round] =  14 / 1000
* [client count] =  1 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.092428
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.081881
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.074341
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.074757
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.073193
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055304
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client1] Train Ep

Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.073528
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.090199
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.076400
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.086675
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.073960
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.081121
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.075834
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.081027
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.077774
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.073875
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.089034
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.8551, Accuracy: 639/753 (85%)

** [total 

...

Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.091348
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.069383
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.113168
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.072064
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.093480
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.097571
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.072325
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.065203
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.065887
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.087619
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9417, Accuracy: 483/753 (64%)

** [total train time]:  7:44:26
** [round train time]:  0:00:29

[round] 

Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.104044
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.071611
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.081320
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.071738
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.077121
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.8875, Accuracy: 644/753 (86%)

** [total train time]:  7:47:18
** [round train time]:  0:00:29

[round] =  977 / 1000
* [client count] =  1 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.087884
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.089675
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.096764
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.085826
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.085061
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.087339
* [client_train_time


Test set: Average loss for Global model: 0.9019, Accuracy: 609/753 (81%)

** [total train time]:  7:50:10
** [round train time]:  0:00:29

[round] =  983 / 1000
* [client count] =  1 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.091357
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.079323
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.086255
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.087523
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.081491
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.056173
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.095078
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.089376
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.083255
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.103655
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.09317

Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.093270
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.085252
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.078982
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.104449
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.087615
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.075803
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.072458
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.090491
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.075258
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.105299
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9216, Accuracy: 606/753 (80%)

** [total train time]:  7:53:31
** [round train time]:  0:00:29

[round] 

Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.079991
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.090443
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.079229
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.075778
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.077627
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9271, Accuracy: 551/753 (73%)

** [total train time]:  7:56:22
** [round train time]:  0:00:29

[round] =  996 / 1000
* [client count] =  1 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.093827
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.096177
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.069559
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.074098
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.065093
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.075294
* [client_train_time

In [14]:
# tensorboard open 
# tensorboard --logdir=/home/citi302/Desktop/Codefolder/FL_DP_covid/runs